In [6]:
import numpy as np
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

input_size = 784
hidden_size = 512
hidden_size2 = 256
batch_size = 100
learning_rate = 0.001
num_classes = 10
num_epochs = 5

# MNIST Dataset 
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),  
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


In [7]:
class FeedForwardNN(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size2, output_size):
        super(FeedForwardNN, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size2)
        self.linear3 = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        l1 = F.relu(self.linear1(x))
        l2 = F.relu(self.linear2(l1))
        out = self.linear3(l2)
        return out
    

NN = FeedForwardNN(input_size, hidden_size, hidden_size2, num_classes)
NN.cuda(0)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(NN.parameters(), lr=learning_rate)

In [10]:
# training
for epoch in range(num_epochs):
    for i, (image, labels) in enumerate(train_loader):
        images = Variable(image.view(-1, 28*28).cuda())
        labels = Variable(labels.cuda())
        
        optimizer.zero_grad()
        output = NN(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        if(i+1) % 100 ==0:
            print('epoch: [%d/%d], step: [%d/%d], Loss: %.4f' % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

epoch: [1/5], step: [100/600], Loss: 0.0398
epoch: [1/5], step: [200/600], Loss: 0.0530
epoch: [1/5], step: [300/600], Loss: 0.0458
epoch: [1/5], step: [400/600], Loss: 0.0425
epoch: [1/5], step: [500/600], Loss: 0.0765
epoch: [1/5], step: [600/600], Loss: 0.0564
epoch: [2/5], step: [100/600], Loss: 0.0331
epoch: [2/5], step: [200/600], Loss: 0.0464
epoch: [2/5], step: [300/600], Loss: 0.0755
epoch: [2/5], step: [400/600], Loss: 0.0083
epoch: [2/5], step: [500/600], Loss: 0.0919
epoch: [2/5], step: [600/600], Loss: 0.0728
epoch: [3/5], step: [100/600], Loss: 0.0110
epoch: [3/5], step: [200/600], Loss: 0.0329
epoch: [3/5], step: [300/600], Loss: 0.0171
epoch: [3/5], step: [400/600], Loss: 0.0054
epoch: [3/5], step: [500/600], Loss: 0.0150
epoch: [3/5], step: [600/600], Loss: 0.0675
epoch: [4/5], step: [100/600], Loss: 0.0058
epoch: [4/5], step: [200/600], Loss: 0.0496
epoch: [4/5], step: [300/600], Loss: 0.0388
epoch: [4/5], step: [400/600], Loss: 0.1961
epoch: [4/5], step: [500/600], L

In [11]:
#test
total = 0
correct = 0

for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28).cuda())
    _, predicted = torch.max(NN(images).data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()    

print('accuracy of the model : %d %%' % (100*correct/total))

accuracy of the model : 97 %
